---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [16]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split

# import seaborn as sns
# import matplotlib.pyplot as plt
# %matplotlib inline
def blight_model():
    test_df = pd.read_csv('test.csv',encoding = 'ISO-8859-1')
    train_df=pd.read_csv('train.csv',encoding = 'ISO-8859-1')
    ad_df=pd.read_csv('addresses.csv',encoding = 'ISO-8859-1').set_index('address')
    ll_df=pd.read_csv('latlons.csv',encoding = 'ISO-8859-1').set_index('address')
    al_df=pd.merge(ad_df,ll_df,how='inner',left_index=True,right_index=True).reset_index().set_index('ticket_id')
    
    t_df=train_df[pd.notnull(train_df['compliance'])].set_index('ticket_id')
    t_df=t_df[['disposition','ticket_issued_date','hearing_date','fine_amount','late_fee','discount_amount','judgment_amount','compliance']]
    
    tt_df=test_df.set_index('ticket_id')
    tt_df=tt_df[['disposition','ticket_issued_date','hearing_date','fine_amount','late_fee','discount_amount','judgment_amount']]
    
    t_df=pd.merge(t_df,al_df,how='inner',left_index=True,right_index=True)
    tt_df=pd.merge(tt_df,al_df,how='inner',left_index=True,right_index=True)
# #     sns.heatmap(t_df.isnull(),cbar=False,yticklabels=False,cmap = 'viridis')
#     t_df.grafitti_status.replace(to_replace={np.NaN: 0,'GRAFFITI TICKET':1}, inplace=True)
#     tt_df.grafitti_status.replace(to_replace={np.NaN: 0,'GRAFFITI TICKET':1}, inplace=True)
    
    t_df.disposition.replace(to_replace={'Responsible by Default': 4,'Responsible by Admission':3,'Responsible by Determination':2,'Responsible (Fine Waived) by Deter':1}, inplace=True)
    tt_df.disposition.replace(to_replace={'Responsible by Default': 4,'Responsible by Admission':3,'Responsible by Determination':2,'Responsible (Fine Waived) by Deter':1,
        'Responsible (Fine Waived) by Admis':0,'Responsible - Compl/Adj by Default':0,'Responsible - Compl/Adj by Determi':0,'Responsible by Dismissal':0}, inplace=True)
    
    t_df.ticket_issued_date=pd.to_datetime(t_df.ticket_issued_date)
    t_df.hearing_date=pd.to_datetime(t_df.hearing_date)
    t_df['time']=(t_df['hearing_date'].dt.date-t_df['ticket_issued_date'].dt.date).dt.days
    m=t_df[~(t_df['time']<0)].time.mean()
    t_df.time.fillna(value=int(m),inplace=True)
    t_df.time=t_df.time.apply(lambda x : x if x>=0 else int(m))
    
    tt_df.ticket_issued_date=pd.to_datetime(tt_df.ticket_issued_date)
    tt_df.hearing_date=pd.to_datetime(tt_df.hearing_date)
    tt_df['time']=(tt_df['hearing_date'].dt.date-tt_df['ticket_issued_date'].dt.date).dt.days
    m=tt_df[~(tt_df['time']<0)].time.mean()
    tt_df.time.fillna(value=int(m),inplace=True)
    tt_df.time=tt_df.time.apply(lambda x : x if x>=0 else int(m))
    
    t_df.drop(['address','hearing_date','ticket_issued_date'],axis=1,inplace=True)
    tt_df.drop(['address','hearing_date','ticket_issued_date'],axis=1,inplace=True)

    l1=t_df.lat.mean()
    l2=t_df.lon.mean()
    t_df.lat.fillna(value=l1,inplace=True)
    t_df.lon.fillna(value=l2,inplace=True)
    
    l1=tt_df.lat.mean()
    l2=tt_df.lon.mean()
    tt_df.lat.fillna(value=l1,inplace=True)
    tt_df.lon.fillna(value=l2,inplace=True)
    
    X_train=t_df.drop(['compliance'],axis=1)
    y_train=t_df['compliance']
#     X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state = 0,stratify=y)

    clf = GradientBoostingClassifier(random_state = 0).fit(X_train, y_train)
#     y_score = clf.decision_function(X_test)
    y_pp=clf.predict_proba(tt_df)[:,1]
    s=pd.Series(y_pp,index=tt_df.index.values.tolist())#.reindex(tt_df.index)
#     fpr, tpr, _ = roc_curve(y_test, y_score)
#     roc_auc = auc(fpr, tpr)
#     fi=clf.feature_importances_
#     s1=clf.score(X_train, y_train)
#     s2=clf.score(X_test, y_test)
    return s

In [17]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


328722    0.031849
350971    0.031849
344821    0.031849
358133    0.071101
309054    0.057368
286216    0.017444
324319    0.077113
355752    0.034733
314897    0.251483
349403    0.247133
303732    0.049925
303731    0.057368
327945    0.033859
330126    0.236558
371204    0.193382
371203    0.236558
288692    0.077113
288756    0.029546
309575    0.077113
311449    0.041558
311472    0.028109
311474    0.063176
324028    0.077113
290001    0.028483
290002    0.019962
290000    0.028483
327364    0.060515
311292    0.076596
339649    0.243574
325836    0.047399
            ...   
307465    0.036721
307457    0.018719
311528    0.016187
343394    0.045884
343393    0.014838
343395    0.045061
341673    0.045884
320985    0.035962
335819    0.022816
335818    0.036721
335820    0.063201
337265    0.063201
337279    0.035962
337267    0.022338
348237    0.063201
357114    0.022338
357113    0.035962
357115    0.054067
328744    0.063201
375646    0.025834
291325    0.053388
300550    0.